# Defining agents using ScaDS.AI's LLM service
In this notebook we will program a simple agent using a remote institutional server and the OpenAI API. For executing this notebook, you need an environment variable named `SCADSAI_API_KEY`. You can get such a key [here](https://llm.scads.ai/).

In [2]:
import os
import openai
import json

These helper functions are stored in [function_calling_scadsai.py](function_calling_scadsai.py) and not shown for the readers convenience.

In [10]:
from function_calling_scadsai import function_to_dict, act, call_function_from_response

In [5]:
tools = []

@tools.append
def generate_random_number() -> int:
    """Generate a random number between 0 and 100"""
    return random.randint(0, 100)

@tools.append
def roberts_operator(a:int, b:int) -> int:
    """Apply Robert's operator to a and b"""
    return a * b + 1

In [60]:
CONTINUE_PROMPT = "\n\nPlease repeat your question if I should rephrase the results of the recent tool calls."
TOOLCALL_PROMPT = "\n\nCall tools if necessary, but don't do it if the answer is already above."
CONTINUE_PHRASE = "Let me continue to work on this"

def prompt_scadsai_llm(message, tool_dicts=[], endpoint:str= "https://llm.scads.ai/v1", model:str="meta-llama/Llama-3.3-70B-Instruct", verbose=False, history=[]):
    """
    Submit a prompt to a remotely running model and returns the response.
    """
    import os

    api_key = os.environ.get('SCADSAI_API_KEY')
    
    import openai
    client = openai.OpenAI(base_url=endpoint,
                       api_key=api_key)

    next_message = {"role": "user", "content": message}

    print("Q:", history + [next_message])

    completion = client.chat.completions.create(
        model=model,
        messages=history + [next_message],
        tools=tool_dicts,
    )
    print("A:", completion)
        
    return completion


def call_function_from_response(tool_calls, named_tools, verbose=False):
    """
    Takes a response-string from ollama/mistral raw mode, extracts a function to be called and its parameters and calls the function.
    The function must be in the dictionary named_tools where keys are names of functions and values are the corresponding functions.
    """
    import json

    if len(tool_calls) == 0:
        raise ValueError("Not a function call")

    results = []
    for tool_call in tool_calls:
        func = tool_call.function
        func_name = func.name
        arguments = json.loads(func.arguments)

        result = named_tools[func_name](**arguments)
        results.append(result)
    return results

class Agent():
    def __init__(self, tools, verbose=True):
        self._tools = tools
        self._history = []
        self._verbose = verbose

    def act(self, prompt:str, max_iterations=3):
        """
        Takes a prompt and the configured list of function tools, and turns it into a prompt to a function-calling capable language model.
        If the model can select a function to call considering the prompt, this function will be called.
        """
        tools = self._tools
        
        # get a dictionary with names of functions as keys, and corresponding functions as values
        named_tools = {f.__name__: f for f in tools}
        
        # convert the functions to json
        tool_dicts = [function_to_dict(f) for f in tools] 

        if self._verbose:
            print("User:", prompt)
        
        # submit the prompt
        completion = prompt_scadsai_llm(prompt + TOOLCALL_PROMPT, tool_dicts, history=self._history)
        result = completion.choices[0].message.content
        if self._verbose:
            print("Assistant:", result)

        if completion.choices[0].message.tool_calls is not None:
            # call the function as specified in the response
            tool_calls = completion.choices[0].message.tool_calls
            tool_results = call_function_from_response(tool_calls, named_tools)
            
            self._history.append({"role": "user", "content": prompt})
            #self._history.append({"role": "assistant", "content": result})
            tool_call_message = ["I had to call some function tool(s) to answer your request:"]
            for tool_call, tool_result in zip(tool_calls, tool_results):
                func = tool_call.function
                func_name = func.name
                arguments = func.arguments

                tool_call_message.append(f"* I called {func_name} ({arguments}) and the result was: {tool_result}")

                if self._verbose:
                    print("Tool call:", func_name, arguments, "=", tool_result)

            response = "\n".join(tool_call_message)
            self._history.append({"role": "assistant", "content": response})

            if max_iterations > 0:        
                final_phrase = f"\n\nAnswer with '{CONTINUE_PHRASE}' (exactly like this) if you cannot find the answer yet."
            else:
                final_phrase = ""
            continue_answer = prompt_scadsai_llm(f"What is the ultimate answer to the following request given the discussion and tool calls we had so far?\n\n{prompt}{final_phrase}",
                                                    history=self._history,
                                                    ).choices[0].message.content
            
            if max_iterations > 0 and CONTINUE_PHRASE.lower() in continue_answer.lower():
                return self.act(prompt, max_iterations=max_iterations-1)   
                
            return continue_answer
        else:
            self._history.append({"role": "user", "content": prompt})
            self._history.append({"role": "assistant", "content": result})
            return result
    

In [61]:
a = Agent(tools=tools)
a.act("What is the results of Robert's operator on two random numbers?")

User: What is the results of Robert's operator on two random numbers?
Q: [{'role': 'user', 'content': "What is the results of Robert's operator on two random numbers?\n\nCall tools if necessary, but don't do it if the answer is already above."}]
A: ChatCompletion(id='d746eeac-f7b6-485e-aa43-32696a058c38', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='0', function=Function(arguments='{"b": 24, "a": 42}', name='roberts_operator', description=None), type='function')]))], created=1734542942, model='meta-llama/Llama-3.3-70B-Instruct', object='chat.completion', service_tier=None, system_fingerprint='3.0.1-sha-bb9095a', usage=CompletionUsage(completion_tokens=25, prompt_tokens=437, total_tokens=462, completion_tokens_details=None, prompt_tokens_details=None))
Assistant: None
Tool call: roberts_operator {"b": 24, "a": 42} = 10

"The results of Robert's operator on two random numbers is 1009"

In [62]:
a.act("What is the results of Robert's operator on 3 and 5?")

User: What is the results of Robert's operator on 3 and 5?
Q: [{'role': 'user', 'content': "What is the results of Robert's operator on two random numbers?"}, {'role': 'assistant', 'content': 'I had to call some function tool(s) to answer your request:\n* I called roberts_operator ({"b": 24, "a": 42}) and the result was: 1009'}, {'role': 'user', 'content': "What is the results of Robert's operator on 3 and 5?\n\nCall tools if necessary, but don't do it if the answer is already above."}]
A: ChatCompletion(id='600b51af-9474-4c17-81f5-90ddceebf03e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='0', function=Function(arguments='{"b": 5, "a": 3}', name='roberts_operator', description=None), type='function')]))], created=1734542946, model='meta-llama/Llama-3.3-70B-Instruct', object='chat.completion', service_tier=None, syste

"The result of Robert's operator on 3 and 5 is 16"

## Defining function tools
The following two functions will serve as functions the model can call. We also define some image memory, we can manage using the functions.

In [4]:
image_memory = {}

In [ ]:
# Create an agent that uses the OpenAI GPT-4o model.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="YOUR_API_KEY",
)
agent = AssistantAgent(
    name="fantasy_math_expert",
    model_client=model_client,
    tools=[generate_random_number, roberts_operator],
    system_message="Use tools to solve tasks.",
)